In [ ]:
import pandas as pd
import datacompy
import sqlite3

In [ ]:
INPUT_FOLDER = "inputs"
OUTPUT_FOLDER = "outputs"
DB_PATH = f"{INPUT_FOLDER}/quran_backup.db"
OUTPUT_FILE = f"{OUTPUT_FOLDER}/page_history.csv"
OUTPUT_FILE_XL = f"{OUTPUT_FOLDER}/page_history_test.xlsx"

### 
Utils

In [ ]:
def read_multiple_tables(db_path, table_names):
    conn = sqlite3.connect(db_path)
    tables = {}
    
    for table_name in table_names:
        tables[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    
    conn.close()
    return tables

Create a connection to the SQLite database

In [ ]:
table_names = ['hafizs_items']
tables = read_multiple_tables(DB_PATH, table_names)

In [ ]:
# To exclude the inactive items from the revisions records
conn = sqlite3.connect(DB_PATH)
revisions_df = pd.read_sql_query("""
    SELECT t1.*
    FROM revisions t1 
    JOIN items t2 ON t1.item_id = t2.id
    WHERE t2.active = 1
""", conn)
conn.close()

In [ ]:
# Convert date columns to datetime
revisions_df['revision_date'] = pd.to_datetime(revisions_df['revision_date'])
revisions_df.head()


In [ ]:
hafizs_items_df = tables['hafizs_items']
# Convert date columns to datetime
hafizs_items_df['srs_start_date'] = pd.to_datetime(hafizs_items_df['srs_start_date'])
hafizs_items_df.head()

In [ ]:
# We are only generating the history for hafiz = 1 (Siraj)
revisions_df = revisions_df[revisions_df['hafiz_id'] == 1]
hafizs_items_df = hafizs_items_df[hafizs_items_df['hafiz_id'] == 1]

Based on the logic on `README.md`, we can generate the page history table using the following function

In [ ]:
# Claude generated v2

def create_transition_record(hafiz_id, item_id, date, from_status, to_status, from_mode, to_mode, reps, graduated_by):
    """
    Create a standardized transition record
    """
    # Temprorary adding the pages column here based on the item_id, to debug
    pages = hafizs_items_df[hafizs_items_df['item_id'] == item_id]['page_number'].values[0]

    # If `from` and `to` are the same, set Null
    if from_status == to_status:
        from_status = None
        to_status = None
    if from_mode == to_mode:
        from_mode = None
        to_mode = None

    return {
        'hafiz_id': hafiz_id,
        'item_id': item_id,
        'pages': pages,
        'date': date,
        'from_status': from_status,
        'to_status': to_status,
        'from_mode': from_mode,
        'to_mode': to_mode,
        'reps': reps,
        'graduated_by': graduated_by
    }


def calculate_reps_and_graduation(from_mode, from_status, to_mode, item_revisions, current_revision_index=None):
    """
    Calculate reps and graduated_by for transitions FROM specified modes
    """
    
    # Rule 1: If from_status is 6, graduated_by = user
    if from_status == 6:
        reps = None
        if from_mode in [3, 4, 5]:
            mode_count = len(item_revisions[item_revisions['mode_id'] == from_mode])
            denominator = 7 if from_mode in [3, 4] else 11
            reps = f"{mode_count}/{denominator}"
        return reps, 'user'
    
    # Rule 2: If from_mode is 2, graduated_by = system
    if from_mode == 2:
        return None, 'system'
    
    # Rule 3: If to_mode is 5, check previous two revision ratings
    if to_mode == 5 and current_revision_index is not None:
        # Get the two revisions before the current transition
        if current_revision_index >= 2:
            prev_two_ratings = item_revisions.iloc[current_revision_index-2:current_revision_index]['rating'].tolist()
            # If both previous ratings are bad (-1), then system, else user
            if len(prev_two_ratings) == 2 and all(rating == -1 for rating in prev_two_ratings):
                graduated_by = 'system'
            else:
                graduated_by = 'user'
        else:
            # Not enough previous records, default to user
            graduated_by = 'user'
        
        return None, graduated_by
    
    # Original logic for other cases
    if from_mode not in [3, 4, 5]:
        return None, None
    
    # Count revisions under the from_mode for this item
    mode_count = len(item_revisions[item_revisions['mode_id'] == from_mode])
    
    # Set denominator based on mode
    denominator = 7 if from_mode in [3, 4] else 11  # mode 5 = 11
    
    # Format reps and determine graduation
    reps = f"{mode_count}/{denominator}"
    graduated_by = 'system' if mode_count >= denominator else 'user'
    
    return reps, graduated_by

def generate_page_history(revisions_df, hafizs_items_df):
    """
    Generate page_history table based on revisions and current item status
    """
    # Initialize empty list to store page history records
    page_history_records = []
    
    # Mode to Status mapping
    mode_status_mapping = {1: 1, 2: 4, 3: 4, 4: 4, 5: 5}
    
    # Process each unique combination of hafiz_id and item_id
    for (hafiz_id, item_id) in revisions_df.groupby(['hafiz_id', 'item_id']).groups.keys():
        # Get revisions for this hafiz-item combination, sorted by date
        item_revisions = revisions_df[
            (revisions_df['hafiz_id'] == hafiz_id) & 
            (revisions_df['item_id'] == item_id)
        ].sort_values('revision_date')
        
        # Initialize starting state
        current_mode = None
        current_status = 6  # Initial status is 6
        
        # Process each revision chronologically
        for idx, (_, revision) in enumerate(item_revisions.iterrows()):
            # Calculate new state
            new_mode = revision['mode_id']
            new_status = mode_status_mapping.get(new_mode, new_mode)
            
            # Only create record if mode has changed
            if current_mode != new_mode:
                # Calculate reps and graduated_by using modular function
                reps, graduated_by = calculate_reps_and_graduation(
                    current_mode, current_status, new_mode, item_revisions, idx
                )
                
                # Determine the date to use
                date_to_use = revision['revision_date']
                
                # Special case: if transitioning TO mode 5, check hafizs_items
                if new_mode == 5:
                    hafizs_item = hafizs_items_df[
                        (hafizs_items_df['hafiz_id'] == hafiz_id) & 
                        (hafizs_items_df['item_id'] == item_id)
                    ]
                    # If srs_start_date is not null, use it
                    srs_start_date = hafizs_item.iloc[0].get('srs_start_date')
                    if pd.notna(srs_start_date):
                        date_to_use = srs_start_date
                
                # Create and add transition record
                record = create_transition_record(
                    hafiz_id, item_id, date_to_use, 
                    current_status, new_status, 
                    current_mode, new_mode, 
                    reps, graduated_by
                )
                page_history_records.append(record)
                
                # Update current state for next iteration
                current_mode = new_mode
                current_status = new_status
        
        # Check if final state matches hafizs_items current state
        hafizs_item = hafizs_items_df[
            (hafizs_items_df['hafiz_id'] == hafiz_id) & 
            (hafizs_items_df['item_id'] == item_id)
        ]
        
        if not hafizs_item.empty:
            current_hafizs_mode = hafizs_item.iloc[0]['mode_id']
            current_hafizs_status = hafizs_item.iloc[0]['status_id']
            
            # If final revision state doesn't match hafizs_items, add final transition
            if current_mode != current_hafizs_mode or current_status != current_hafizs_status:
                # Calculate reps and graduated_by using modular function
                reps, graduated_by = calculate_reps_and_graduation(
                    current_mode, current_status, current_hafizs_mode, item_revisions
                )
                
                # Determine the date to use for final transition
                date_to_use = item_revisions.iloc[-1]['revision_date']
                
                # Special case: if transitioning TO mode 5 and hafizs_items also has mode 5
                srs_start_date = hafizs_item.iloc[0].get('srs_start_date')
                if pd.notna(srs_start_date):
                    date_to_use = srs_start_date
                
                # Create and add final transition record
                final_record = create_transition_record(
                    hafiz_id, item_id, date_to_use,
                    current_status, current_hafizs_status,
                    current_mode, current_hafizs_mode,
                    reps, graduated_by
                )
                page_history_records.append(final_record)
    
    # Convert to DataFrame
    return pd.DataFrame(page_history_records)

In [ ]:
item_id = 548
mock_revisions_df = revisions_df[revisions_df['item_id'] == item_id]
mock_hafizs_items_df = hafizs_items_df[hafizs_items_df['item_id'] == item_id]

generate_page_history(mock_revisions_df, mock_hafizs_items_df)

In [ ]:
item_id = 104
mock_revisions_df = revisions_df[revisions_df['item_id'] == item_id]
mock_hafizs_items_df = hafizs_items_df[hafizs_items_df['item_id'] == item_id]

generate_page_history(mock_revisions_df, mock_hafizs_items_df)

In [ ]:
page_history_df = generate_page_history(revisions_df, hafizs_items_df)



In [ ]:
page_history_df.dtypes

In [ ]:
page_history_df = page_history_df.astype({
    'hafiz_id': 'int64',
    'item_id': 'int64', 
    'pages': 'int64',
    'date': 'datetime64[ns]',
    'from_status': 'Int64',
    'to_status': 'Int64',
    'from_mode': 'Int64',
    'to_mode': 'Int64',
    'reps': 'object',
    'graduated_by': 'object',
})



In [ ]:
page_history_df.to_csv(OUTPUT_FILE, index=False)


generate_page_history(revisions_df, hafizs_items_df).to_excel(OUTPUT_FILE_XL, index=False)

Compare the old logic with the new logic

In [ ]:
df1 = pd.read_csv(OUTPUT_FOLDER+'/page_history_old.csv')

df2 = pd.read_csv(OUTPUT_FOLDER+'/page_history.csv')

compare = datacompy.Compare(
df1, df2, join_columns='id'
)

In [ ]:
print(compare.report())